In [9]:
import json
import pandas as pd
import requests
from collections import OrderedDict

searchman_key_tomo = ""  # 23/100
apptweak_key_tomo = ""  # 0/100
searchman_key_masha = ""  # 0/100

In [58]:
# Google play metadata
df_metadata = pd.read_csv("data/googleplay_metadata_phr.csv"); df_metadata.shape

(20, 8)

In [59]:
app_names = list(df_metadata['name'])
app_ids = list(df_metadata['id'])

In [28]:
# Use Searchman API to get up to 100 reviews

def get_reviews(names, ids, sortby):
    sort = sortby
    count = 100

    app_reviews_all = {}
    for app_name, app_id in list(zip(names, ids)):
        print(app_id)
        url = "http://api.searchman.io/v1/android/us/app/reviews?" \
            "appId=%s&sort=%s&count=%s&apiKey=%s" % (app_id, sort, str(count), searchman_key_tomo)
        r = requests.get(url)
        reviews = json.loads(r.text)
        app_reviews_all[app_name] = reviews['data']

com.upmc.healthplan
com.humetrix.iBlueButton
com.ecw.healow
com.sharecare.realgreen
com.blueshieldca.prod
com.higi.main
com.cigna.mobile.mycigna
com.medfusion.claire
com.wellpoint.consumer.ABCBS
com.fitnesssyncer.fitnesssyncer_for_samsung_health
com.teladoc.members
com.usablenet.android.aetna
org.kp.m
com.humana.myhumana
com.accolade.mobile.prod
com.jardogs.fmhmobile
com.google.android.apps.fitness
vladimir.yerokhin.medicalrecord
com.hyrax.mymedical
com.cliniconline


In [29]:
app_reviews_helpful = get_reviews(app_names, app_ids, "mosthelpful")
app_reviews_recent = get_reviews(app_names, app_ids, "mostrecent")

{'UPMC Health Plan': [{'id': 1540901737520,
   'rating': 5,
   'body': 'This app is very useful and works well. Be sure you made a MyHealth OnLine account through a browser first, then use that account you created at the Login on the Application.',
   'reviewId': 'gp:AOqpTOGCyQPcWRlsQR0w2ZmyAE-LDzyhcLwWsiGCNgdPLMEiK-wlzY-mvj-kV6Sig0W4c4--pm_mvMOeHSBPZw',
   'version': '18.2',
   'authorId': '107612558926338329616',
   'author': 'Noah Camp',
   'authorProfileImageUrl': 'https://lh5.googleusercontent.com/-P5RCrXLht8Q/AAAAAAAAAAI/AAAAAAAAAAA/ABtNlbAY9d-1EU2G908HQ_uYeLdPWcNmow/photo.jpg',
   'timestampEpoch': 1540901737,
   'timestamp': '2018-10-30 12:15:37'},
  {'id': 1540407634630,
   'rating': 1,
   'body': "I couldn't even create an account. Keeps saying there is an error",
   'reviewId': 'gp:AOqpTOFsl8hFtND0-ZZhCQ12BUVQQ1q2XkF_5TC4X1JSyu1Q4zQzTIiVle_kLRzgUOdJ6qxt4kh94BBYwhuHhA',
   'version': '18.2',
   'authorId': '112619492728795859109',
   'author': 'becky sowers',
   'authorProfil

In [38]:
with open("googleplay_reviews_recent_raw.json", "w") as f:
    f.write(json.dumps(app_reviews_recent))
    
with open("googleplay_reviews_helpful_raw.json", "w") as f:
    f.write(json.dumps(app_reviews_helpful))

In [39]:
# Aggregate helpful and recent reviews and 
# create a unique list of reviews for each app.

app_reviews_together = {}
for app in app_names:
    helpful_review_ids = [review['id'] for review in app_reviews_helpful[app]]
    recent_review_ids = [review['id'] for review in app_reviews_recent[app]]
    all_reviews_ids = list(set(helpful_review_ids).union(set(recent_review_ids)))
    app_reviews_all = app_reviews_helpful[app] + app_reviews_recent[app]
    app_reviews_unique = list({review['id']:review for review in app_reviews_all}.values())
    app_reviews_together[app] = app_reviews_unique

In [67]:
# json to tabular format.

app_reviews_tabular = []
for ix, metadata in df_metadata.iterrows():
    reviews = app_reviews_together[metadata['name']]
    for review in reviews:
        row = OrderedDict()
        row['name'] = metadata['name']
        row['id'] = metadata['id']
        row['userRatingCount'] = metadata['userRatingCount']
        row['avgUserRating'] = metadata['avgUserRating']
        row['title'] = review.get('title')
        row['author'] = review.get('author')
        row['authorId'] = review.get('authorId')
        row['rating'] = review.get('rating')
        row['date'] = review.get('timestamp')
        row['voteSum'] = review.get('voteSum')
        row['voteCount'] = review.get('voteCount')
        row['content'] = review.get('body')
        if row['content'] is not None:
            row['content'] = row['content'].replace('\n', '')
        app_reviews_tabular.append(row)

In [91]:
df_out = pd.DataFrame(app_reviews_tabular); df_out.head(2)

,name,id,userRatingCount,avgUserRating,title,author,authorId,rating,date,voteSum,voteCount,content
0,UPMC Health Plan,com.upmc.healthplan,199,2.61809,None,Noah Camp,107612558926338329616,5,2018-10-30 12:15:37,None,None,This app is very useful and works well. Be sur...
1,UPMC Health Plan,com.upmc.healthplan,199,2.61809,None,becky sowers,112619492728795859109,1,2018-10-24 19:00:34,None,None,I couldn't even create an account. Keeps sayin...


In [86]:
df_out = df_out[df_out['content'].isna() == False]  # Remove reviews with no content

In [89]:
df_out.to_csv("data/googleplay_reviews_phr.csv")